<a href="https://colab.research.google.com/github/NguyenThuan215/int3404-3-simpleORC/blob/main/cnn_simpleORC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from collections import namedtuple
from sklearn.metrics import classification_report

import torch
from torch import nn
from torch.utils.data import DataLoader

In [2]:
!git clone https://github.com/NguyenThuan215/int3404-3-simpleORC/

Cloning into 'int3404-3-simpleORC'...
remote: Enumerating objects: 671, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 671 (delta 1), reused 21 (delta 1), pack-reused 649
Receiving objects: 100% (671/671), 654.80 MiB | 31.80 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (637/637), done.


In [3]:
traindir = "/content/int3404-3-simpleORC/data/train"
testdir = "/content/int3404-3-simpleORC/data/test"
modeldir = "/content/int3404-3-simpleORC/model.pth"

In [4]:
transform_train = transforms.Compose([
                                  transforms.Resize((256,256)),
                                  transforms.RandomCrop(224, padding=4),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                  ])

transform_test = transforms.Compose([
                                  transforms.Resize((224,224)),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                  ])

TrainTest = namedtuple('TrainTest', ['train', 'test'])
classes = ['highlands', 'others', 'phuclong', 'starbucks']
def prepare_data():
  trainset = torchvision.datasets.ImageFolder(root=traindir, transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root=testdir, transform=transform_test)
  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  batch = 8
  worker = 4
  trainloader = DataLoader(dataset=datasets.train, batch_size=batch, shuffle=True, num_workers=worker)
  testloader = DataLoader(dataset=datasets.test, batch_size=batch, shuffle=False, num_workers=worker)
  return TrainTest(train=trainloader, test=testloader)
  
def imshow(img):
    img = img / 2 + 0.5 
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [5]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  final_loss = 0.0
  reporting_steps = 10
  step = 0
  for images, labels in loader:
    step += 1
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    final_loss = loss.item()
    running_loss += final_loss
    if step % reporting_steps == reporting_steps - 1:
      print(f"Epoch {epoch} step {step} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0
  return final_loss

def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    for images, labels in loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue

In [6]:
def main():
  PATH = "./model.pth"
  class_out = len(classes)

  datasets = prepare_data()
  loaders = prepare_loader(datasets)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  print("image size: ", datasets.test[0][0].shape)
  print("len train: ", len(datasets.train))
  print("len test: ", len(datasets.test))
  print("len batch train: ", len(loaders.train))
  print("device:", device)

  model = torchvision.models.resnet18(pretrained=True)
  model.fc = torch.nn.modules.linear.Linear(in_features=512, out_features=class_out, bias=True)
  model.to(device)

  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

  accuracies = []
  losses = []
  for epoch in range(40):
    print("---------------------------------------------------------------")
    loss = train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ypred_test, ytrue_test = test_epoch(epoch, model, loaders.test, device)

    print(classification_report(ytrue_test, ypred_test, target_names=classes))
    torch.save(model.state_dict(), PATH)

    # calculate report
    ypred_test = np.array(ypred_test)
    ytrue_test = np.array(ytrue_test)
    accuracy = int((ytrue_test==ypred_test).sum() / len(ytrue_test) *100)
    accuracies.append(accuracy)
    losses.append(round(loss, 4))

  print("accr: ", accuracies)
  print("loss: ", losses)
  return model

In [ ]:
datasets = prepare_data()
loader = prepare_loader(datasets)
imshow(datasets.test[21][0])
print(datasets.test[21][1])
print("class:", datasets.train.classes)
model = main()

In [65]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

modeldir = "/content/int3404-3-simpleORC/model.pth"
sample_dir = '/content/int3404-3-simpleORC/sampledata'

#load model
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.modules.linear.Linear(in_features=512, out_features=4, bias=True)
model.to(device)
model.load_state_dict(torch.load(modeldir, map_location=device))
model.to(device)

#transforms
img_transforms = transforms.Compose([
                                transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                ])

images = os.listdir(sample_dir) #list of image
images.sort()

ypred = []
for image_name in images:
  path = os.path.join(sample_dir, image_name)
  image = Image.open(path)
  image = img_transforms(image)
  image = np.transpose(image, (0,1,2))
  image = image.unsqueeze(0) 
  image = image.to(device)

  model.eval()
  with torch.no_grad():
     output = model(image)
     _, predicted = torch.max(output, dim=1)
     ypred += (list(predicted.cpu().numpy()))
     print(image_name, output, classes[int(predicted.cpu().numpy())])

print(ypred)

img1.jpeg tensor([[11.1119, -4.3118, -2.9040, -4.1733]], device='cuda:0') highlands
img10.jpeg tensor([[-6.7432, 14.7271, -6.5645, -1.3590]], device='cuda:0') others
img2.jpeg tensor([[ 6.4072, -4.8636, -1.9358,  0.0890]], device='cuda:0') highlands
img3.jpeg tensor([[11.1119, -4.3118, -2.9040, -4.1733]], device='cuda:0') highlands
img4.jpeg tensor([[-6.9463, -2.6218,  8.9481,  0.4895]], device='cuda:0') phuclong
img5.jpeg tensor([[-3.6445, -5.3688, 11.0620, -2.0510]], device='cuda:0') phuclong
img6.jpeg tensor([[-9.8279, -3.4960, -4.1228, 16.4280]], device='cuda:0') starbucks
img7.jpeg tensor([[-2.8155,  2.4542, -0.6047,  0.8546]], device='cuda:0') others
img8.jpeg tensor([[-0.5924, -0.1753, -0.2259,  0.8041]], device='cuda:0') starbucks
img9.jpeg tensor([[-6.4125, 13.2863, -2.9095, -4.0267]], device='cuda:0') others
[0, 1, 0, 0, 2, 2, 3, 1, 3, 1]
